<a href="https://colab.research.google.com/github/johanhoffman/DD2363_VT24/blob/chmntz_Lab2/chmntz_Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2: Iterative methods**
Carl **Chemntiz**

# **Abstract**
TODO:
* Write abstract
* Write introduction
* Write GMRES
* Write results
* Write discussion

# **Set up environment**
To have access to the necessary modules you have to run this cell.

In [190]:
from google.colab import files

import numpy as np
from numpy.linalg import norm, inv
from scipy.optimize import fsolve

# Iterative methods requires an arbitrary tolerance
TOL = 1e-9

# **Introduction**

Jacobi and Gauss-Seidel iteration will be guaranteed to converge if $A$ is diagonally dominant,
$$|a_{ii}|\geq\sum_{j\neq i}|a_{ij}|\hspace{1em}\forall i$$
and there exists at least one index $k$ for which $a$ is strictly diagonally dominant,
$$|a_{kk}|>\sum_{j\neq k}|a_{kj}|$$

TODO
* Explain Stationary iterative methods
* Matrix splitting

Jacobi iteration is equivalent to left Jacobi preconditioned Richardson iteration with $\alpha=1$.

Gauss-Seidel iteration is equivalent to left preconditioned Richardson iteration with $B=L^{-1}$ and $\alpha=1$.

# **Methods**

## Jacobi iteration for $Ax=b$
Jacobi iteration is an algorithm to approximate solution $x$ to $Ax=b$ where $A$ is a strictly diagonally dominant matrix. The method is based on matrix splitting using a diagonal matrix,
$$A_1=D,\hspace{1em}A_2=A-D$$
where $D=\text{diag}(A)$.
This gives the iteration matrix $M_J$
$$M_J=-L^{-1}(A-L)=I-D^{-1}A$$
and the constant vector $c$
$$c=D^{-1}b$$
Since matrix splitting is a stationary iterative method, we get that the linear iteration

$$x^{(k+1)}=M_Jx^{(k)}+c=(I-D^{-1}A)x^{(k)}+D^{-1}b$$
From chapter 7, example 7.8 presents the Jacobi iteration in terms of the components of $A=(a_{ij})$,

$$x_i^{(k+1)}=a_{ii}^{-1}\Big(b_i-\sum_{j\neq i}a_{ij}x_j^{(k)}\Big),\hspace{1em}\forall i$$
This can be translated into the algorithm:
```
Input: n x n diagonally dominant matrix A, n vector b.
Output: solution vector x
1:  while norm(r)/norm(b) > TOL do
2:    for i=0:n do
3:      row_sum = 0
4:      for j=0:i and j!=i do
5:        row_sum = row_sum + A[i,j]*x_old[j]
6:      end for
7:      x[i] = (b[i] - row_sum)/(A[i,i])
8:    end for
9:  end while
10: return x
```
which was then implemented into Python code.

In [191]:
def jacobi_iteration(A: np.array, b: np.array) -> np.array:
    n = len(b)
    x = np.zeros(n)

    while norm(A @ x - b) / norm(b) > TOL:
        x_k = x.copy()
        for i in range(n):
            a_inv = 1 / A[i,i]
            row_sum = 0
            for j in range(n):
                if j == i: continue
                row_sum += A[i,j] * x_k[j]
            x[i] = a_inv * (b[i] - row_sum)
    return x

However, convergance of an iterative method can be improved by using *relaxation*. Relaxation is an method of setting the degree of diagonal dominance by a parameter $\omega>0$. This gives us the damped Jacobi iteration.
$$x^{(k+1)}=x^{(k)}+\omega D^{-1}(b-Ax^{(k)})$$
The book (chapter 7, section 7) also suggests an technique to improve convergance even further using *successive over-relaxation*, however, this technique was not implemented.

In [192]:
def damped_jacobi_iteration(A: np.array, b: np.array) -> np.array:
    x = b.copy()
    D = np.diag(np.diag(A))
    M = np.eye(len(b)) - inv(D) @ A
    # c = inv(D) @ b
    w = 0.7

    while norm(A @ x - b) / norm(b) > TOL:
        # x = M @ x + c
        x = x + w * inv(D) @ (b - A @ x)
    return x

## Gauss-Seidel iteration for $Ax=b$
Gauss-Seidel iteration is similar to Jacobi iteration, using the same principles but splitting the matrix using the lower triangular matrix. Unlike Jacobi iteration, $A$ does not have to be strictly diagonally dominant, and can be irreducibly diagonally dominant aswell.
$$A_1=L,\hspace{1em}A_2=A-L$$
Similarily the iteration matrix is
$$M_{GS}=-L^{-1}(A-L)=I-L^{-1}A$$
and the vector becomes
$$c=L^{-1}b$$
The form expressed in terms of components is

$$x_i^{(k+1)}=a_{ii}^{-1}\Big(b_i-\sum_{j<i}a_{ij}x_j^{(k+1)}-\sum_{j>i}a_{ij}x_j^{(k)}\Big),\hspace{1em}\forall i$$

In [193]:
def gauss_seidel_iteration(A: np.array, b: np.array) -> np.array:
    n = len(b)
    x = np.zeros(n)

    while norm(A @ x - b) / norm(b) > TOL:
        x_k = x.copy()
        for i in range(n):
            a_inv = 1 / A[i,i]
            row_sum = 0
            for j in range(i):
                row_sum += A[i,j] * x[j]
            for j in range(i+1, n):
                row_sum += A[i,j] * x_k[j]
            x[i] = a_inv * (b[i] - row_sum)
    return x

As Gauss-Seidel iteration is a stationary iterative method, the method can be damped using relaxation.

In [194]:
def damped_gauss_seidel_iteration(A: np.array, b: np.array) -> np.array:
    x = b.copy()
    L = np.tril(A)
    M = np.eye(len(b)) - inv(L) @ A
    c = inv(L) @ b
    w = 0.7

    while norm(A @ x - b) / norm(b) > TOL:
        # x = M @ x + c
        x = x + w * inv(L) @ (b - A @ x)
    return x

## Newton's method for scalar nonlinear equation $f(x)=0$
Newton's method is an iterative algorithm to approxiamte the roots of a function based on an initial guess. The function $f$ must be a differentiable function.
$$x^{(k+1)}=x^{(k)}-\frac{f(x^{(k)})}{f'(x^{(k)})}$$

Since Newton's method requires the derivative of a function, a method to approximate the finite difference was implemented. Central difference was chosen due to its higher accuracy compared to forward and backward differences.

In [195]:
def derivate(f, x: float) -> float:
    h = 1e-15
    df = (f(x + h) - f(x - h)) / (2 * h)
    if df == 0: raise ArithmeticError
    return df

In this lab `newtons_method` was implemented based on algorithm 8.2 from the book *Methods in Computational Science*, which itself was an implementation of the equation stated above.

In [196]:
def newtons_method(f, x: float) -> float:
    while abs(f(x)) > TOL:
        x = x - (f(x) / derivate(f, x))
    return x

## GMRES method for $Ax=b$
TODO: Write

In [197]:
def arnoldi_iteration(A: np.array, b: np.array, k: int) -> np.array:
    Q = np.zeros((A.shape[0], k+1))
    H = np.zeros((k+1, k))
    Q[:,0] = b / norm(b)

    for j in range(1,k+1):
        v_j = A @ Q[:,j-1]
        for i in range(0,j):
            H[i,j-1] = Q[:,i] @ v_j
            v_j -= H[i,j-1] * Q[:,i]
        H[j, j-1] = norm(v_j)
        Q[:, j] = v_j / H[j, j-1]
    return Q, H

The GMRES algorithm itself was implemented based on algorithm 7.2 from the book *Methods in Computational Science*.

In [198]:
def gmres(A: np.array, b: np.array) -> np.array:
    k_max = 1000
    Q = np.zeros((A.shape[0], k_max))
    Q[:,0] = b / norm(b)

    for k in range(1, k_max):
        e1 = np.zeros(k+1)
        e1[0] = 1

        Q, H = arnoldi_iteration(A, b, k)
        y = np.linalg.lstsq(H, norm(b)*e1, rcond=None)[0]
        r = (norm(b) * e1) - (H @ y)

        if norm(r) / norm(b) < TOL: break;
    x = Q[:, 0:k] @ y
    return x

# **Results**

In [199]:
A = np.random.random((5,5)) + 3 * np.eye(5)
b = np.random.rand(5)

## Jacobi iteration for $Ax=b$

Convergance of residuual test $||Ax-b||$.

In [200]:
print(f"||Ax-b||=0: {np.allclose(A @ jacobi_iteration(A,b), b)}")
print(f"||Ax-b||=0: {np.allclose(A @ damped_jacobi_iteration(A,b), b)}")

||Ax-b||=0: True
||Ax-b||=0: True


Difference between manufactured and exact solution, $||x-y||$.

In [201]:
print(f"||x-y||=0: {np.allclose(np.linalg.solve(A,b), jacobi_iteration(A,b))}")
print(f"||x-y||=0: {np.allclose(np.linalg.solve(A,b), damped_jacobi_iteration(A,b))}")

||x-y||=0: True
||x-y||=0: True


## Gauss-Seidel iteration for $Ax=b$

In [202]:
print(f"||Ax-b||=0: {np.allclose(A @ gauss_seidel_iteration(A,b), b)}")
print(f"||Ax-b||=0: {np.allclose(A @ damped_gauss_seidel_iteration(A,b), b)}")

||Ax-b||=0: True
||Ax-b||=0: True


In [203]:
print(f"||x-y||=0: {np.allclose(np.linalg.solve(A,b), gauss_seidel_iteration(A,b))}")
print(f"||x-y||=0: {np.allclose(np.linalg.solve(A,b), damped_gauss_seidel_iteration(A,b))}")

||x-y||=0: True
||x-y||=0: True


## Newton's method for scalar nonlinear equation $f(x)=0$


In [204]:
while True:
    c = np.random.randint(0,9,4)
    f = lambda x : c[3] * x**3 + c[2] * x**2 + c[1] * x + c[0]
    try:
        x = newtons_method(f, 0)
        break
    except ArithmeticError:
        continue

In [205]:
print(f"|f(x)|=0: {np.isclose(f(x),0)}")

|f(x)|=0: True


In [206]:
print(f"|x-y|=0: {np.isclose(x, fsolve(f, 0))[0]}")

|x-y|=0: True


## GMRES methods for $Ax=b$

In [207]:
print(f"||Ax-b||=0: {np.allclose(A @ gmres(A,b), b)}")

||Ax-b||=0: True


In [208]:
print(f"||x-y||=0: {np.allclose(np.linalg.solve(A,b), gmres(A,b))}")

||x-y||=0: True


# **Discussion**